In [1]:
import pandas as pd

In [4]:
pd.read_csv('AutoScout24_3172.csv')

,Unnamed: 0,Car,Price,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,0,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,CHF 18'799.–,05.2012,68'200 km,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
1,1,AUDI RS e-tron GT quattro,CHF 155'990.–,Neues Fahrzeug,9 km,Automatikgetriebe,Elektro,599 PS (440 kW),469 km,4147 Aesch
2,2,BMW 120d xDrive Sport Line,CHF 36'900.–,05.2023,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
3,3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,CHF 21'900.–,03.2016,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal
4,4,MERCEDES-BENZ B 250 e AMG Line 8G-DCT,CHF 37'900.–,11.2020,9'500 km,Halbautomatisches Getriebe,Plug-in hybrid Benzin/Elektro,217 PS (160 kW),1.5 l/100 km,6438 Ibach-Schwyz
...,...,...,...,...,...,...,...,...,...,...
63435,63435,LAND ROVER Range Rover Evoque 2.0 TD4 Pure AT9,CHF 22'890.–,09.2018,124'000 km,Automat,Diesel,150 PS (110 kW),5.1 l/100 km,6370 Stans
63436,63436,HYUNDAI Tucson 1.6 CRDi Vertex 4WD,CHF 26'900.–,05.2019,67'450 km,Halbautomatisches Getriebe,Diesel,136 PS (100 kW),6.6 l/100 km,8400 Winterthur
63437,63437,CITROEN Grand C4 Picasso 2.0 BlueHDi Shine,CHF 13'900.–,08.2016,57'000 km,Schaltgetriebe manuell,Diesel,150 PS (110 kW),4.1 l/100 km,8618 Oetwil am See
63438,63438,AUDI A8 4.2 FSI quattro tiptronic,CHF 15'900.–,11.2010,157'000 km,Automat,Benzin,372 PS (273 kW),9.5 l/100 km,8618 Oetwil am See
